In [1]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Gemini 2.0 text - Single-turn

* [Google Gen AI SDKs](https://cloud.google.com/vertex-ai/generative-ai/docs/sdks/overview)
  * The new Google Gen AI SDK provides a unified interface to Gemini 2.0 through both the Gemini Developer API and the Gemini API on Vertex AI. With a few exceptions, code that runs on one platform will run on both.
  * This means that you can prototype an application using the Developer API and then migrate the application to Vertex AI without rewriting your code.

* [google-genai 0.3.0](https://pypi.org/project/google-genai/)
* [Google Gen AI SDK](https://googleapis.github.io/python-genai/)
* [python-genai : Github](https://github.com/googleapis/python-genai)

## Set configuration

### Package Install
* [google-cloud-aiplatform](https://cloud.google.com/python/docs/reference/aiplatform/latest)

In [2]:
%pip install --upgrade --user --quiet google-genai

### Authentication to access to GCP
* Only for Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.
* Refer to the [authentication ways](https://cloud.google.com/docs/authentication?hl=ko) in GCP

In [3]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab Execution

### Define constants

In [4]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-2.0-flash-exp" # @param {type:"string"}

### Import libraries

In [5]:
import base64

from google import genai
from google.genai.types import (
    Content,
    FunctionDeclaration,
    GenerateContentConfig,
    GenerateContentResponse,
    GoogleSearch,
    Part,
    Retrieval,
    SafetySetting,
    Tool,
    VertexAISearch,
)


### Initalize client

* [genai.Client](https://googleapis.github.io/python-genai/genai.html#genai.client.Client)

In [6]:
client = genai.Client(
    vertexai=True,
    project=PROJECT_ID,
    location=LOCATION
)

### Helper function

In [7]:
def generate_gemini_2_0(query:str) -> GenerateContentResponse:

  contents = [
    Content(
      role="user",
      parts=[
        # https://googleapis.github.io/python-genai/genai.html#genai.types.Part
        # Check class methods. (from_bytes, from_code_execution_result, from_executable_code, from_function_call, etc... )
        Part.from_text(query)
      ]
    )
  ]

  # https://googleapis.github.io/python-genai/genai.html#genai.types.SafetySettingDict
  safety_settings = [SafetySetting(category="HARM_CATEGORY_HATE_SPEECH",threshold="OFF"),
                     SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT",threshold="OFF"),
                     SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT",threshold="OFF"),
                     SafetySetting(category="HARM_CATEGORY_HARASSMENT",threshold="OFF")]

  google_search_tool = Tool(google_search=GoogleSearch())
  tools=[google_search_tool]

  # https://googleapis.github.io/python-genai/genai.html#genai.types.GenerateContentConfig
  generate_content_config = GenerateContentConfig(
    temperature = 1,
    top_p = 0.95,
    max_output_tokens = 8192,
    response_modalities = ["TEXT"],
    safety_settings = safety_settings,
    tools = tools
  )

  # https://googleapis.github.io/python-genai/genai.html#genai.models.AsyncModels.generate_content_stream
  responses = client.models.generate_content_stream(
    model = MODEL_NAME,
    contents = contents,
    config = generate_content_config,
  )

  return responses

In [8]:

prompt = """
        당신은 사람들이 AI 개념을 이해하도록 돕는 생성 AI 전문가입니다.
        아래 내용에 대해서 상세하게 알려주세요.

        1. 최신 생성형 AI 트랜드에 대해서 설명해주세요.
        2. 올해 12월에 구글 생성형 AI 주요 뉴스(날짜 포함)에 대해서 10가지 항목으로 요약해주세요.
        3. 생성형 AI의 2025년도 중요한 주제는 어떤것인가요 ?
"""

responses = generate_gemini_2_0(prompt)

for chunk in responses:
  print(chunk.text, end="")


생성 AI 전문가로서, 사람들이 AI 개념을 이해하도록 돕는 데 최선을 다하겠습니다. 질문하신 내용에 대해 자세히 설명해 드리겠습니다.

**1. 최신 생성형 AI 트렌드**

2024년에는 생성형 AI가 인공지능 분야에서 가장 큰 트렌드로 자리 잡았습니다. 주요 트렌드는 다음과 같습니다:

*   **대중화 및 민주화**: 챗GPT와 같은 텍스트 및 이미지 생성기가 일반 대중에게 공개되면서 전 세계적으로 널리 사용되고 있으며, 기술 지식이 없는 사람들도 AI를 사용할 수 있게 되었습니다.
*   **직장 생산성 향상**: AI는 직장에서의 생산성을 높이는 데 점점 더 중요한 역할을 하고 있습니다. 콘텐츠 생성, 언어 번역, 검색 엔진 등의 작업을 자동화하여 효율성을 높입니다.
*   **산업 전반에 걸친 도입**: AI 기술이 의료, 소매, 금융 등 다양한 산업 분야에서 활용되어 업무 프로세스를 개선하고 있습니다.
*   **창의적인 콘텐츠 생성**: AI는 예술 작품, 음악 작곡, 스토리텔링과 같은 창의적인 콘텐츠를 생성하는 데 사용되며, AI가 창작 과정에서 협력자 역할을 하고 있습니다.
*   **개인화 및 맞춤화**: AI는 개인화된 콘텐츠 추천, 맞춤형 사용자 인터페이스, 개별화된 학습 자료를 제공하여 사용자 중심 애플리케이션에 대한 수요를 충족시키고 있습니다.
*   **멀티모달 기능**: 텍스트, 이미지, 오디오 등 다양한 데이터 조합을 통해 결과물을 생성하는 멀티모달 모델이 발전하고 있습니다. 이러한 모델은 이미지 설명, 창의적인 콘텐츠 생성 등 다양한 작업에 강력한 도구로 활용되고 있습니다.
*   **고급 자연어 처리(NLP)**: AI는 인간의 언어를 보다 정확하고 자연스럽게 이해하고 해석하는 방향으로 빠르게 발전하고 있습니다. 이러한 발전은 고객 서비스에 사용되는 생성 AI 챗봇과 같은 애플리케이션에 큰 영향을 미칩니다.
*   **에이전트 AI**: 에이전트 AI는 대규모 언어 모델(LLM)의 자연어 처리 기능과 다양한 도구를 결합하여